# Tensorboard Advanced

Visualización avanzada mediante Tensorboard (pesos, gradiente,...). Este ejemplo utiliza la base de datos de dígitos manuscritos del MNIST.
(http://yann.lecun.com/exdb/mnist/).

In [ ]:
from __future__ import print_function

import tensorflow as tf

# Importar datos MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [ ]:
# Parametros
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/tmp/tensorflow_logs/example/'

# Network Parametros
n_hidden_1 = 256 # Número de características de la 1ª capa
n_hidden_2 = 256 # Número de características de la 2ª capa
n_input = 784 # MNIST entrada de datos (forma img: 28*28)
n_classes = 10 # MNIST total de clases (0-9 dígitos)

# tf Entrada de gráficos
# datos mnist de imagen de la forma 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# Reconocimiento de 0-9 dígitos => 10 clases
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

In [ ]:
# Crear modelo
def multilayer_perceptron(x, weights, biases):
    # Capa oculta con activación RELU
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Crear un resumen para visualizar la primera capa de activación de ReLU
    tf.summary.histogram("relu1", layer_1)
    # Capa oculta con activación RELU
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Crear otro resumen para visualizar la segunda capa de activación de ReLU
    tf.summary.histogram("relu2", layer_2)
    # Capa de salida
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

# Almacenar capas de peso y sesgo
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

In [ ]:
# Encapsulando todas las operaciones en alcances, haciendo el gráfico de Tensorboard
# Visualización más conveniente
with tf.name_scope('Model'):
    # Construir modelo
    pred = multilayer_perceptron(x, weights, biases)

with tf.name_scope('Loss'):
    # Softmax Cross entropy (función de coste)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

with tf.name_scope('SGD'):
    # Descenso en pendiente
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Op para calcular cada gradiente variable
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    # Op para actualizar todas las variables según su gradiente
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)

with tf.name_scope('Accuracy'):
    # Precisión
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

In [ ]:
# Inicializar las variables (es decir, asignar su valor por defecto)
init = tf.global_variables_initializer()

# Crear un resumen para monitorear el tensor de costos
tf.summary.scalar("loss", loss)
# Crear un resumen para monitorear el tensor de precisión
tf.summary.scalar("accuracy", acc)
# Crear resúmenes para visualizar los pesos
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
# Resumir todos los degradados
for grad, var in grads:
    tf.summary.histogram(var.name + '/gradient', grad)
# Fusionar todos los resúmenes en una sola operación
merged_summary_op = tf.summary.merge_all()

In [ ]:
# Iniciar Entrenamiento
with tf.Session() as sess:

    # Ejecutar el inicializador
    sess.run(init)

    # op para escribir registros en la Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path,
                                            graph=tf.get_default_graph())

    # Ciclo de entrenamiento
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Circuito sobre todos los lotes
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Ejecutar optimización op (backprop), cost op (para obtener valor de pérdida)
            # y nodos de resumen
            _, c, summary = sess.run([apply_grads, loss, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Escribe registros en cada iteración
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Calcula la pérdida promedio
            avg_cost += c / total_batch
        # Visualizar logs por paso de época
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test modelo
    # Calcular la precisión
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch: 0001 cost= 59.570364205
Epoch: 0002 cost= 13.585465186
Epoch: 0003 cost= 8.379069252
Epoch: 0004 cost= 6.005265894
Epoch: 0005 cost= 4.498054792
Epoch: 0006 cost= 3.503682522
Epoch: 0007 cost= 2.822272765
Epoch: 0008 cost= 2.306899852
Epoch: 0009 cost= 1.912765543
Epoch: 0010 cost= 1.597006118
Epoch: 0011 cost= 1.330172869
Epoch: 0012 cost= 1.142490618
Epoch: 0013 cost= 0.939443911
Epoch: 0014 cost= 0.820920588
Epoch: 0015 cost= 0.702543302
Epoch: 0016 cost= 0.604815631
Epoch: 0017 cost= 0.505682561
Epoch: 0018 cost= 0.439700446
Epoch: 0019 cost= 0.378268929
Epoch: 0020 cost= 0.299557848
Epoch: 0021 cost= 0.269859066
Epoch: 0022 cost= 0.230899029
Epoch: 0023 cost= 0.183722090
Epoch: 0024 cost= 0.164173368
Epoch: 0025 cost= 0.142141250
Optimization Finished!
Accuracy: 0.9336
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser


### Visualización de Pérdidas y Precisión
<img src="../../resources/img/tensorboard_advanced_1.png"/>

### Visualización de gráficos de computación
<img src="../../resources/img/tensorboard_advanced_2.png"/>

### Visualización de Pesos y Gradientes
<img src="../../resources/img/tensorboard_advanced_3.png"/>

### Activaciones Visualización
<img src="../../resources/img/tensorboard_advanced_4.png"/>